# Pathway selection and network-file creation
In this notebook we will select pathways from 3 databases, i.e. [KEGG](https://www.genome.jp/kegg/), [REACTOME](https://reactome.org/) and [WikiPathways](https://www.wikipathways.org/index.php/WikiPathways).
We base this selection on genes associated with inflammation, of which the retrieving process is described in a different [notebook](https://github.com/macsbio/inflammation_networks/tree/master/Jupyter%20-%20Inflammation%20genes).

#### The following step only works in RStudio. If working in another environment, please set the working directory properly and check if the working directory is correct. 

In [27]:
# set wd to where script file is saved
setwd(dirname(rstudioapi::callFun("getActiveDocumentContext")$path))

ERROR: Error: RStudio not running


In [1]:
# check wd
getwd()

[1] "C:/Users/Laurent/Documents/GitHub/inflammation_networks2/Jupyter - Pathway selection network file creation"

In [2]:
# load library
library(limma)
library(qusage)
library(plyr)
library(dplyr)
library(tidyr)

Warning message:
"package 'qusage' was built under R version 3.5.2"
Attaching package: 'dplyr'

The following objects are masked from 'package:plyr':

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'tidyr' was built under R version 3.5.2"

First we will load in the .gmt files of the 3 databases, next we will change the structure so we can work more easily with it based on our needs. Next we will load in the genes associated with inflammation.

In [3]:
# load data files and clean data files
### KEGG
kegg <- read.gmt(file.path(getwd(), "databases", "c2.cp.kegg.v6.2.entrez.gmt"))
kegg <- ldply(kegg, data.frame)
colnames(kegg)[c(1,2)] <- c("pathway", "entrezgene")

In [4]:
### REACTOME
reactome <- read.gmt(file.path(getwd(), "databases", "c2.cp.reactome.v6.2.entrez.gmt"))
reactome <- ldply(reactome, data.frame)
colnames(reactome)[c(1,2)] <- c("pathway", "entrezgene")

In [5]:
### WikiPathways
wp <- read.gmt(file.path(getwd(), "databases", "wikipathways-combined_final.gmt"))
wp <- ldply(wp, data.frame)
colnames(wp)[c(1,2)] <- c("pathway", "entrezgene")

Warning message in readLines(file):
"incomplete final line found on 'C:/Users/Laurent/Documents/GitHub/inflammation_networks2/Jupyter - Pathway selection network file creation/databases/wikipathways-combined_final.gmt'"

In [6]:
# combine databases
allDatabases <- rbind(kegg, reactome, wp)

In [7]:
# load in gene list associated with inflammation
infl <- as.data.frame(read.table(file.path(getwd(), "genes", "merged_infl_genes.txt"), header = T, sep = "\t", stringsAsFactors = FALSE))
infl <- as.data.frame(infl[,c(-1,-3,-4)])
colnames(infl)[1] <- "entrezgene"
head(infl)

entrezgene
50616
1236
847
6647
5327
361


Now the 3 databases and the genes associated with inflammation are loaded in, we can start with filtering the pathways based on our criteria and the genes associated with inflammation. 

The criteria are; the pathways has to contain at least 10 genes, has to contain at least 5 genes associated with inflammation and at least 30% of the genes in the pathway have to be associated with inflammation.

In [8]:
# filter first that pathway has at least 10 genes, so the row with the pathwayname has to occur at least 10 times in the data frame
allDatabases <- allDatabases[allDatabases$pathway %in% names(table(allDatabases$pathway))[table(allDatabases$pathway) >= 10],]
# freq of every pathway in database
freqallDatabases <- plyr::count(allDatabases$pathway)
# pathway names of the rows (entrezgene IDs) that occur in the gene list associated with inflammation
allDatabasespathways <- as.data.frame(allDatabases$pathway[allDatabases$entrezgene %in% infl$entrezgene])
# if these pathwaynames occur at least 5 times we can create list of pathways with at least 5 inflammation genes 
# (and which were already filtered on at least 10 gesn in total)
colnames(allDatabasespathways)[1] <- "pathway"
allDatabasespathways <- as.data.frame(allDatabasespathways[allDatabasespathways$pathway %in% names(table(allDatabasespathways$pathway))[table(allDatabasespathways$pathway) >= 5],])
colnames(allDatabasespathways)[1] <- "pathway"
# freq of every pathway in allDatabasespathways
freqallDatabasespathways <- plyr::count(allDatabasespathways$pathway)
# freqallDatabases has total amount of genes in the pathway and freqallDatabasespathways has total amount of infalmmation genes in pathway
# combine the two data frames, so we have the total amount of genes per pathway which has at least 5 inflammation genes in them
# and so we also have the amount of inflammation genes per pathway, so we can calculate the percentage, which has to be at least 30%
allDatabasespathways <- as.data.frame(merge(freqallDatabasespathways, freqallDatabases, by = "x"))
colnames(allDatabasespathways)[c(1,2,3)] <- c("pathway", "freqInflGene", "freqGene")
# calculate percentage and add column
# select rows which have percentage of at least 30%
allDatabasespathways$percentage <- ((allDatabasespathways$freqInflGene/allDatabasespathways$freqGene) * 100)
allDatabasespathways <- allDatabasespathways[allDatabasespathways$percentage >= 30, ]
allDatabasespathways$percentage <- round(allDatabasespathways$percentage, 2)

Now we have for each database the selected pathways based on our criteria. Lets save those with the stats of how many genes the pathway has, how many genes associated with inflammation and the percentage of genes associated with inflammation within the pathway. 

Next, 3 pathways were manually added, while they did not meet the criteria, as they were seen as highly important for the inflammation network.

In [9]:
# combine the retrieved pathways so we have a list of the pathways that met the criteria
pathwayStats <- as.data.frame(allDatabasespathways)

# save pathwaysStats
write.table(pathwayStats, file.path(getwd(), "results", "pathwayStats.txt"), col.names = T, row.names = F, sep = "\t", quote = F)

# get list of only selected pathways without stats
selectedpws <- pathwayStats[c(-2,-3,-4)]  

In [10]:
# Three pathways were manually added although they did not meet the criteria, lets add them to the selectedpws
added <- data.frame(c("IL17 signaling pathway%WikiPathways_20181110%WP2112%Homo sapiens", 
                      "IL-6 signaling pathway%WikiPathways_20181110%WP364%Homo sapiens",
                      "Relationship between inflammation, COX-2 and EGFR%WP4483%Homo sapiens"))
colnames(added)[1] <- "pathway"
selectedpws <- rbind(selectedpws, added)

# save selectedpws
write.table(selectedpws, file.path(getwd(), "results", "selected_pws.txt"), col.names = T, row.names = F, sep = "\t", quote = F)

Now we have a list of selected pathways, it is interesting to see in how many pathways the inflammation genes are.
First we will check in how many total pathways the genes are, then check in how many selected pathways these genes are.

In [11]:
# in how many pathways are the inflammation genes
nGenesTotal <- as.data.frame(allDatabases$entrezgene[allDatabases$entrezgene %in% infl$entrezgene])
colnames(nGenesTotal)[1] <- "entrezgene"

# in how many slected pathways are the inflammation genes
selectedallDatabases <- as.data.frame(allDatabases[allDatabases$pathway %in% allDatabasespathways$pathway,])
nInflGenesTotal <- as.data.frame(selectedallDatabases$entrezgene[selectedallDatabases$entrezgene %in% infl$entrezgene])
colnames(nInflGenesTotal)[1] <- "entrezgene"

In [12]:
# combine all frequencies together
# first the inflammation genes in all pathways
nGenesTotalF <- plyr::count(nGenesTotal)
colnames(nGenesTotalF)[2] <- "n_All_Pathways"
# next the inflammation genes in selected pathways
nInflGenesTotalF <- plyr::count(nInflGenesTotal)
colnames(nInflGenesTotalF)[2] <- "n_Sel_Pathways"

In [13]:
# save tables
write.table(nGenesTotalF, file.path(getwd(), "results", "nGenes.txt"), col.names = T, row.names = F, sep = "\t", quote = F)
write.table(nInflGenesTotalF, file.path(getwd(), "results", "nInflGenes.txt"), col.names = T, row.names = F, sep = "\t", quote = F)

Now we have the selected pathways, we will create the network files which will be used in the Cytoscape automation using the RCy3 package. 

In [14]:
# edge table
edge_table <- as.data.frame(allDatabases[allDatabases$pathway %in% selectedpws$pathway,])
head(edge_table)

,pathway,entrezgene
7430,KEGG_NOD_LIKE_RECEPTOR_SIGNALING_PATHWAY,7205
7431,KEGG_NOD_LIKE_RECEPTOR_SIGNALING_PATHWAY,841
7432,KEGG_NOD_LIKE_RECEPTOR_SIGNALING_PATHWAY,1147
7433,KEGG_NOD_LIKE_RECEPTOR_SIGNALING_PATHWAY,257397
7434,KEGG_NOD_LIKE_RECEPTOR_SIGNALING_PATHWAY,2919
7435,KEGG_NOD_LIKE_RECEPTOR_SIGNALING_PATHWAY,8767


In [15]:
# cluster pathways together into groups
# Allograft rejection
edge_table$pathway[edge_table$pathway == "KEGG_ALLOGRAFT_REJECTION"] <- "Allograft_rejection"
edge_table$pathway[edge_table$pathway == "Allograft Rejection%WikiPathways_20181110%WP2328%Homo sapiens"] <- "Allograft_rejection"
edge_table$pathway[edge_table$pathway == "KEGG_GRAFT_VERSUS_HOST_DISEASE"] <- "Allograft_rejection"
edge_table$pathway[edge_table$pathway == "Control of immune tolerance by vasoactive intestinal peptide%WP4484%Homo sapiens"] <- "Allograft_rejection"
edge_table$pathway[edge_table$pathway == "Inflammatory Response Pathway%WikiPathways_20181110%WP453%Homo sapiens"] <- "Allograft_rejection"

# Cytokines
edge_table$pathway[edge_table$pathway == "IL-6 signaling pathway%WikiPathways_20181110%WP364%Homo sapiens"] <- "Cytokines"
edge_table$pathway[edge_table$pathway == "IL-10 Anti-inflammatory Signaling Pathway%WP4495%Homo sapiens"] <- "Cytokines"
edge_table$pathway[edge_table$pathway == "Cytokines and Inflammatory Response%WikiPathways_20181110%WP530%Homo sapiens"] <- "Cytokines"
edge_table$pathway[edge_table$pathway == "IL17 signaling pathway%WikiPathways_20181110%WP2112%Homo sapiens"] <- "Cytokines"
edge_table$pathway[edge_table$pathway == "Signal transduction through IL1R%WP4496%Homo sapiens"] <- "Cytokines"
edge_table$pathway[edge_table$pathway == "ncRNAs involved in STAT3 signaling in hepatocellular carcinoma%WikiPathways_20181110%WP4337%Homo sapiens"] <- "Cytokines"

# Diseases
edge_table$pathway[edge_table$pathway == "KEGG_TYPE_I_DIABETES_MELLITUS"] <- "Diseases"
edge_table$pathway[edge_table$pathway == "KEGG_ASTHMA"] <- "Diseases"
edge_table$pathway[edge_table$pathway == "Lung fibrosis%WikiPathways_20181110%WP3624%Homo sapiens"] <- "Diseases"
edge_table$pathway[edge_table$pathway == "KEGG_INTESTINAL_IMMUNE_NETWORK_FOR_IGA_PRODUCTION"] <- "Diseases"

# Immune cell regulation
edge_table$pathway[edge_table$pathway == "Cells and Molecules involved in local acute inflammatory response%WP4493%Homo sapiens"] <- "Immune_cell_regulation"
edge_table$pathway[edge_table$pathway == "Selective expression of chemokine receptors during T-cell polarization%WP4494%Homo sapiens"] <- "Immune_cell_regulation"
edge_table$pathway[edge_table$pathway == "KEGG_LEISHMANIA_INFECTION"] <- "Immune_cell_regulation"

# NFkB
edge_table$pathway[edge_table$pathway == "KEGG_NOD_LIKE_RECEPTOR_SIGNALING_PATHWAY"] <- "NFkB"
edge_table$pathway[edge_table$pathway == "REACTOME_TAK1_ACTIVATES_NFKB_BY_PHOSPHORYLATION_AND_ACTIVATION_OF_IKKS_COMPLEX"] <- "NFkB"
edge_table$pathway[edge_table$pathway == "REACTOME_TRAF6_MEDIATED_NFKB_ACTIVATION"] <- "NFkB"
edge_table$pathway[edge_table$pathway == "REACTOME_RIP_MEDIATED_NFKB_ACTIVATION_VIA_DAI"] <- "NFkB"
edge_table$pathway[edge_table$pathway == "Photodynamic therapy-induced NF-kB survival signaling%WikiPathways_20181110%WP3617%Homo sapiens"] <- "NFkB"
edge_table$pathway[edge_table$pathway == "Simplified Depiction of MYD88 Distinct Input-Output Pathway%WikiPathways_20181110%WP3877%Homo sapiens"] <- "NFkB"
edge_table$pathway[edge_table$pathway == "EBV LMP1 signaling%WikiPathways_20181110%WP262%Homo sapiens"] <- "NFkB"

# Vitamin B12
edge_table$pathway[edge_table$pathway == "Folate Metabolism%WikiPathways_20181110%WP176%Homo sapiens"] <- "Vitamin_B12"
edge_table$pathway[edge_table$pathway == "Vitamin B12 Metabolism%WikiPathways_20181110%WP1533%Homo sapiens"] <- "Vitamin_B12"
edge_table$pathway[edge_table$pathway == "IL1 and megakaryocytes in obesity%WikiPathways_20181110%WP2865%Homo sapiens"] <- "Vitamin_B12"

# Inflammation
edge_table$pathway[edge_table$pathway == "Vitamin D in inflammatory diseases%WP4482%Homo sapiens"] <- "Inflammation"
edge_table$pathway[edge_table$pathway == "LTF danger signal response pathway%WP4478%Homo sapiens"] <- "Inflammation"
edge_table$pathway[edge_table$pathway == "Resistin as a regulator of inflammation%WP4481%Homo sapiens"] <- "Inflammation"
edge_table$pathway[edge_table$pathway == "Relationship between inflammation, COX-2 and EGFR%WP4483%Homo sapiens"] <- "Inflammation"

In [16]:
# save edge table
write.table(edge_table, file.path(getwd(), "results", "edge_table.txt"), col.names = T, row.names = F, sep = "\t", quote = F)

In [18]:
# node table
# split edge table in two seperate data frames, rbind, unique, add type.
pathwayNodes <- as.data.frame(unique(edge_table$pathway))
colnames(pathwayNodes)[1] <- "nodes"
geneNodes <- as.data.frame(unique(edge_table$entrezgene))
colnames(geneNodes)[1] <- "nodes"
node_table <- as.data.frame(rbind(pathwayNodes, geneNodes))
# add type
node_table$Type <- "Gene"
node_table$Type[node_table$nodes %in% infl$entrezgene] <- "InflGene"
node_table$Type[node_table$nodes %in% selectedpws$pathway] <- "Process"
node_table$Type[node_table$nodes == "Allograft_rejection"] <- "Process"
node_table$Type[node_table$nodes == "Cytokines"] <- "Process"
node_table$Type[node_table$nodes == "Diseases"] <- "Process"
node_table$Type[node_table$nodes == "Immune_cell_regulation"] <- "Process"
node_table$Type[node_table$nodes == "NFkB"] <- "Process"
node_table$Type[node_table$nodes == "Vitamin_B12"] <- "Process"
node_table$Type[node_table$nodes == "Inflammation"] <- "Process"


# save node table
write.table(node_table, file.path(getwd(), "results", "node_table.txt"), col.names = T, row.names = F, sep = "\t", quote = F)